# INST414 Final Project Sprint 2

In [2]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datascience import *
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Data Cleaning

In [3]:
# select columns to read and keep
cols_to_keep = ['Data_Value', 'Race/Ethnicity', 'Age(months)', 'Sex', 'LocationDesc', 'LocationAbbr', 'YearStart', 'YearEnd', 'Low_Confidence_Limit', 'High_Confidence_Limit', 'Sample_Size', 'GeoLocation']
df = pd.read_csv("/Users/virginialee/Downloads/WIC_data.csv", usecols=cols_to_keep)

df.head()
df.shape

(12852, 12)

In [4]:
# rename columns to be more intuitive
df.rename(columns={
    'Data_Value': 'pct_overweight',
    'Race/Ethnicity': 'race',
    'Age(months)': 'age_months'
}, inplace=True)
df.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,Sex,race,GeoLocation
0,2008,2008,AL,Alabama,15.3,14.7,15.8,"18,219",24 - 35,NaN,NaN,"(32.84057112200048, -86.63186076199969)"
1,2008,2008,AL,Alabama,14.9,14.4,15.5,"14,796",36 - 47,NaN,NaN,"(32.84057112200048, -86.63186076199969)"
2,2008,2008,AL,Alabama,16.4,15.6,17.1,"10,272",48 - 59,NaN,NaN,"(32.84057112200048, -86.63186076199969)"
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,NaN,American Indian/Alaska Native,"(32.84057112200048, -86.63186076199969)"
4,2008,2008,AL,Alabama,8.8,5.4,12.2,273,NaN,NaN,Asian/Pacific Islander,"(32.84057112200048, -86.63186076199969)"


In [5]:
# checking missing values
missing_value = df.isna().sum()
print(missing_value)

YearStart                    0
YearEnd                      0
LocationAbbr                 0
LocationDesc                 0
pct_overweight             379
Low_Confidence_Limit       379
High_Confidence_Limit      379
Sample_Size                379
age_months                9072
Sex                      10584
race                      7182
GeoLocation                  0
dtype: int64


In [165]:
# calculate % missing for key columns
379/12852 # 2.9% missing for data_value
379/12852 # 2.9% missing for low_confidence_limit
379/12852 # 2.9% missing for high_confidence_limit
379/12852 # 2.9% missing for sample_size
7182/12852 # 55.88% missing for race/ethnicity
9072/12852 # 70.59% missing for age
10584/12852 # 82.35% missing for sex

0.8235294117647058

In [6]:
# dropping missing values and unnecessary columns
clean_df = df.dropna(subset=['pct_overweight', 'Low_Confidence_Limit', 'High_Confidence_Limit', 'Sample_Size'])
clean_df = clean_df.drop(columns=['Sex'])
clean_df.head()
clean_df.shape

(12473, 11)

In [7]:
# filter race by American Indian/Alaska Native and Non-Hispanic White
df_race = clean_df[clean_df['race'].isin(['American Indian/Alaska Native', 'Non-Hispanic White'])]
df_race.shape
df_race.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,race,GeoLocation
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,American Indian/Alaska Native,"(32.84057112200048, -86.63186076199969)"
9,2008,2008,AL,Alabama,15.8,15.2,16.3,"17,833",NaN,Non-Hispanic White,"(32.84057112200048, -86.63186076199969)"
14,2008,2008,AK,Alaska,23.7,22.2,25.2,"3,190",NaN,American Indian/Alaska Native,"(64.84507995700051, -147.72205903599973)"
20,2008,2008,AK,Alaska,15.5,14.3,16.7,"3,540",NaN,Non-Hispanic White,"(64.84507995700051, -147.72205903599973)"
25,2008,2008,AZ,Arizona,20.1,17.7,22.4,"1,101",NaN,American Indian/Alaska Native,"(34.865970280000454, -111.76381127699972)"


In [8]:
# duplicates in data set
print(df_race.duplicated().value_counts()) # how many rows are duplicates
df_race[df_race.duplicated(keep=False)] # displays duplicate rows
# drop duplicates
df_race = df_race.drop_duplicates()
df_race.shape

False    2021
True        8
Name: count, dtype: int64


(2021, 11)

In [9]:
# relabel values to be more intuitive
df_race = df_race.replace({
    'American Indian/Alaska Native': 'Indigenous',
    'Non-Hispanic White': 'White'
}, inplace=False)
df_race.head()

/var/folders/19/jdc_mhnj0kb5ql3ygtht4_0m0000gn/T/ipykernel_8709/1362604716.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_race = df_race.replace({


,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,race,GeoLocation
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,Indigenous,"(32.84057112200048, -86.63186076199969)"
9,2008,2008,AL,Alabama,15.8,15.2,16.3,"17,833",NaN,White,"(32.84057112200048, -86.63186076199969)"
14,2008,2008,AK,Alaska,23.7,22.2,25.2,"3,190",NaN,Indigenous,"(64.84507995700051, -147.72205903599973)"
20,2008,2008,AK,Alaska,15.5,14.3,16.7,"3,540",NaN,White,"(64.84507995700051, -147.72205903599973)"
25,2008,2008,AZ,Arizona,20.1,17.7,22.4,"1,101",NaN,Indigenous,"(34.865970280000454, -111.76381127699972)"


In [10]:
# turn race into boolean variable
df_race.drop(columns='Indigenous', errors='ignore', inplace=True)  # drop if already exists
df_race['Indigenous'] = (df_race['race'] == 'Indigenous').astype(int)
df_race.head()
df_race.to_csv('cleaned_wic_data.csv', index=False)

## Baseline Linear Regression Model

In [ ]:
# Fit the linear regression model using sklearn
X = df_race[['Indigenous']]
y = df_race['pct_overweight']
reg = LinearRegression()
reg.fit(X, y)

In [ ]:
reg.coef_  # coefficient for Indigenous variable
reg.intercept_  # intercept
reg.score(X, y)  # R-squared value
print("Coefficient for Indigenous:", reg.coef_[0], "Intercept:", reg.intercept_, "R-squared:", reg.score(X, y))

In [ ]:
# using statsmodels to get p-value
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

## Primary Model

In [ ]:
# writing a function to remove commas for Sample_Size column
def remove_commas(df, column_name):
    if ',' in df[column_name]:
            df[column_name] = df[column_name].str.replace(',', '', regex=True)
    else:
         df[column_name] = df[column_name]
    return df

# testing function on a copy of df_race
test_fx = df_race.copy()
test_fx.head()

remove_commas(test_fx, 'Sample_Size')

,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,race,GeoLocation,Indigenous
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,Indigenous,"(32.84057112200048, -86.63186076199969)",1
9,2008,2008,AL,Alabama,15.8,15.2,16.3,17833,NaN,White,"(32.84057112200048, -86.63186076199969)",0
14,2008,2008,AK,Alaska,23.7,22.2,25.2,3190,NaN,Indigenous,"(64.84507995700051, -147.72205903599973)",1
20,2008,2008,AK,Alaska,15.5,14.3,16.7,3540,NaN,White,"(64.84507995700051, -147.72205903599973)",0
25,2008,2008,AZ,Arizona,20.1,17.7,22.4,1101,NaN,Indigenous,"(34.865970280000454, -111.76381127699972)",1
...,...,...,...,...,...,...,...,...,...,...,...,...
12790,2020,2020,WV,West Virginia,12.6,11.7,13.7,4281,NaN,White,"(38.66551020200046, -80.71264013499967)",0
12796,2020,2020,WI,Wisconsin,22.9,19.3,27.0,445,NaN,Indigenous,"(44.39319117400049, -89.81637074199966)",1
12802,2020,2020,WI,Wisconsin,13.7,12.9,14.5,6844,NaN,White,"(44.39319117400049, -89.81637074199966)",0
12808,2020,2020,WY,Wyoming,15.3,8.8,25.3,72,NaN,Indigenous,"(43.23554134300048, -108.10983035299967)",1


In [ ]:
# applying function to df_race
remove_commas(df_race, 'Sample_Size')

In [17]:
# Independent Variables
# converting Sample_Size and pct_overweight to numeric for statsmodel
df_race['Sample_Size'] = pd.to_numeric(df_race['Sample_Size'], errors='coerce')
df_race['pct_overweight'] = pd.to_numeric(df_race['pct_overweight'], errors='coerce')

X = df_race[['Indigenous']]
X = sm.add_constant(X)

# Dependent Variables
y = df_race['pct_overweight']

# Fit weighted linear regression using sample sizes as weights
model = sm.WLS(y, X, weights=df_race['Sample_Size'])
results = model.fit()

print(results.summary())

                            WLS Regression Results                            
Dep. Variable:         pct_overweight   R-squared:                       0.117
Model:                            WLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     268.2
Date:                Sat, 06 Dec 2025   Prob (F-statistic):           1.08e-56
Time:                        16:10:49   Log-Likelihood:                -6365.1
No. Observations:                2021   AIC:                         1.273e+04
Df Residuals:                    2019   BIC:                         1.275e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.2658      0.063    210.060      0.0

### Robustness Check

In [104]:
# different subsample - years 2014-2020
df_years = df_race[df_race['YearStart'] >= 2014]
df_years.head()

# age variable
df_years['age_months'] = df_years['age_months'].fillna('Missing')
age_dummy = pd.get_dummies(df_years['age_months'], prefix='age', drop_first=True)
age_dummy = age_dummy.astype(int)

X = pd.concat([df_years['Indigenous'], age_dummy], axis=1)
X = sm.add_constant(X)
y = df_years['pct_overweight']

model = sm.WLS(y, X, weights=df_years['Sample_Size'])
results = model.fit()

print(results.summary())

                            WLS Regression Results                            
Dep. Variable:         pct_overweight   R-squared:                       0.104
Model:                            WLS   Adj. R-squared:                  0.103
Method:                 Least Squares   F-statistic:                     132.0
Date:                Sat, 29 Nov 2025   Prob (F-statistic):           5.63e-29
Time:                        17:21:44   Log-Likelihood:                -3534.8
No. Observations:                1137   AIC:                             7074.
Df Residuals:                    1135   BIC:                             7084.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.9738      0.083    156.262      0.0

/var/folders/19/jdc_mhnj0kb5ql3ygtht4_0m0000gn/T/ipykernel_59187/985899154.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_years['age_months'] = df_years['age_months'].fillna('Missing')


## Model Evaluation and Diagnostics

In [ ]:
# checking for independence of residuals
residuals = results.resid

plt.figure(figsize=(8,6))
plt.plot(residuals)
plt.title('Residuals vs Observation from Weighted Linear Regression Model')
plt.xlabel('Observation')
plt.ylabel('Residuals')
plt.savefig('residuals_weighted_linear_regression.png')

In [112]:
# checking for homoskedasticity
fitted = results.fittedvalues

plt.figure(figsize=(8,6))
plt.scatter(fitted, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.savefig('residuals_vs_fitted_weighted_linear_regression.png')

In [114]:
# checking for normality of residuals
sns.histplot(residuals, kde=True)
plt.title('Histogram of Residuals from Weighted Linear Regression Model')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.savefig('histogram_residuals_weighted_linear_regression.png')

In [21]:
# Cook's model for influential points
model = sm.OLS(y, X).fit()
influence = model.get_influence()
cooks_d, _ = influence.cooks_distance


# plot Cook's distance
plt.figure(figsize=(10, 6))
plt.stem(cooks_d)
plt.axhline(4/len(cooks_d), color='red', linestyle='--', label='Threshold: 4/n')
plt.title("Cook's Distance for Each Observation")
plt.ylabel("Cook's Distance")
plt.xlabel("Observation Index")
plt.legend()
plt.savefig('cooks_distance_weighted_linear_regression.png')


In [22]:
# identifying influential points from Cook's distance
influential_points = np.where(cooks_d > 4/len(cooks_d))[0]
print("Influential points (indices):", influential_points)

Influential points (indices): [   0   34   36   40   99  102  120  122  147  158  183  202  204  212  223
  241  252  280  283  284  287  306  340  374  396  414  416  420  450  454
  482  488  496  504  506  516  536  552  576  580  583  585  613  631  684
  690  693  713  723  734  766  790  798  800  804  808  825  835  871  873
  874  883  900  984 1000 1025 1057 1082 1090 1092 1098 1126 1165 1166 1170
 1174 1274 1292 1369 1377 1384 1397 1401 1421 1425 1441 1443 1460 1555 1557
 1560 1577 1579 1631 1664 1671 1700 1842 1850 1858 1869 1900 1941 1967 1993
 2002 2012]


In [24]:
influential_points.shape

(107,)

In [161]:
df_race.to_csv('cleaned_wic_data.csv', index=False)